In [3]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from hepunits import units as u, constants as c

In [301]:
# Get decay rates and daughters of isotopes

import urllib.request
from functools import lru_cache
import re

@lru_cache(maxsize=None)
def lc_pd_dataframe(**kwargs):
    url = "https://nds.iaea.org/relnsd/v1/data?"
    url += "&".join((f"{k}={v}" for k,v in kwargs.items()))
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

In [310]:
all_ground_states = lc_pd_dataframe(fields="ground_states", nuclides="all")

@lru_cache(maxsize=None)
def get_decay_rates(nuclide):
    A, symbol = re.fullmatch("(\d+)(\S+)", nuclide).groups()
    A = int(A)
    ground_state = all_ground_states.query(f"n + z == {A} & symbol == '{symbol}'").iloc[0]
    half_life = ground_state[f"half_life_sec"] * u.s
    if np.isnan(half_life):
        half_life = 1e-99 * u.s
    Z = ground_state["z"]
    N = ground_state["n"]
    daughters = {}
    total_ratio = 0
    for i_d in range(1,4):
        decay_mode = ground_state[f"decay_{i_d}"]
        if not isinstance(decay_mode, str):
            break
        decay_ratio = ground_state[f"decay_{i_d}_%"]
        if np.isnan(decay_ratio):
            decay_ratio = 1e-99
            
        if "B-" in decay_mode:
            d_n = N - 1
            d_z = Z + 1
        elif "B+" in decay_mode or "EC" in decay_mode:
            d_n = N + 1
            d_z = Z - 1
        elif "A" in decay_mode:
            d_n = N - 2
            d_z = Z - 2
        elif "N" in decay_mode:
            d_n = N - 1
            d_z = Z
        elif "P" in decay_mode:
            d_n = N
            d_z = Z - 1
        else:
            break
            
        d_symbol = all_ground_states.query(f"n == {d_n} & z == {d_z}").iloc[0]["symbol"]
        decay = {"d_symbol": d_symbol, "d_n": d_n, "d_z": d_z }
        
        dsym = decay["d_symbol"]
        dA = decay["d_n"] + decay["d_z"]
        daughters[f"{dA}{dsym}"] = decay_ratio
        total_ratio += decay_ratio
    
    for k in daughters:
        # Normalise branching ratios
        daughters[k] /= total_ratio
        # Multiply with decay rate
        daughters[k] *= (np.log(2) / half_life)
    
    return daughters

In [311]:
get_decay_rates("4Li")

{'3He': 6.931471805599453e+89}

In [87]:
states = lc_pd_dataframe(fields="ground_states", nuclides="all")

In [76]:
states.columns

Index(['z', 'n', 'symbol', 'radius', 'unc_r', 'abundance', 'unc_a',
       'energy_shift', 'energy', 'unc_e', 'ripl_shift', 'jp', 'half_life',
       'operator_hl', 'unc_hl', 'unit_hl', 'half_life_sec', 'unc_hls',
       'decay_1', 'decay_1_%', 'unc_1', 'decay_2', 'decay_2_%', 'unc_2',
       'decay_3', 'decay_3_%', 'unc_3', 'isospin', 'magnetic_dipole', 'unc_md',
       'electric_quadrupole', 'unc_eq', 'qbm', 'unc_qb', 'qbm_n', 'unc_qbmn',
       'qa', 'unc_qa', 'qec', 'unc_qec', 'sn', 'unc_sn', 'sp', 'unc_sp',
       'binding', 'unc_ba', 'atomic_mass', 'unc_am', 'massexcess', 'unc_me',
       'me_systematics', 'discovery', 'ENSDFpublicationcut-off',
       'ENSDFauthors', 'Extraction_date'],
      dtype='object')

In [92]:
states[["symbol", "z", "n", "abundance", "energy", "half_life_sec", "decay_1", "decay_1_%", "decay_2", "decay_2_%", "decay_3", "decay_3_%"]]

,symbol,z,n,abundance,energy,half_life_sec,decay_1,decay_1_%,decay_2,decay_2_%,decay_3,decay_3_%
0,n,0,1,NaN,0.0,6.139000e+02,B-,100.0,NaN,NaN,NaN,NaN
1,n,0,4,NaN,0.0,1.754760e-22,NaN,NaN,NaN,NaN,NaN,NaN
2,n,0,6,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H,1,0,99.9855,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H,1,1,0.0145,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3381,Lv,116,177,NaN,0.0,9.500000e-02,A,100.0,SF,12.5,NaN,NaN
3382,Lv,116,178,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3383,Ts,117,176,NaN,0.0,2.100000e-02,A,100.0,SF,6.0,NaN,NaN
3384,Ts,117,177,NaN,0.0,5.100000e-02,A,100.0,NaN,NaN,NaN,NaN


In [93]:
states["decay_1"].unique()

array(['B-', nan, 'N', '2N', 'P', 'B-A', 'A', 'EC', 'EC+B+', '2P', 'B+P',
       'B+', 'B-N', 'B-2N', '2B-', 'ECP', '2EC', '2B+', 'IT', 'SF',
       'ECSF'], dtype=object)

In [289]:
states[states["decay_1"] == "P"][["symbol", "z", "n", "abundance", "energy", "half_life_sec", "decay_1", "decay_1_%", "decay_2", "decay_2_%", "decay_3", "decay_3_%"]]

,symbol,z,n,abundance,energy,half_life_sec,decay_1,decay_1_%,decay_2,decay_2_%,decay_3,decay_3_%
18,Li,3,1,NaN,0.0,NaN,P,100.0,NaN,NaN,NaN,NaN
19,Li,3,2,NaN,0.0,3.709250e-22,P,NaN,NaN,NaN,NaN,NaN
38,B,5,2,NaN,0.0,5.695852e-22,P,100.0,NaN,NaN,NaN,NaN
40,B,5,4,NaN,0.0,8.448847e-19,P,100.0,NaN,NaN,NaN,NaN
67,N,7,3,NaN,0.0,1.824951e-22,P,100.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2435,Ir,77,88,NaN,0.0,1.000000e-06,P,NaN,A,NaN,NaN,NaN
2520,Au,79,91,NaN,0.0,2.900000e-04,P,89.0,A,11.0,NaN,NaN
2521,Au,79,92,NaN,0.0,2.200000e-05,P,100.0,NaN,NaN,NaN,NaN
2608,Tl,81,95,NaN,0.0,5.200000e-03,P,100.0,NaN,NaN,NaN,NaN


In [240]:
nuc = "14C"

In [241]:
alpha = lc_pd_dataframe(fields="decay_rads", nuclides=nuc, rad_types="a")

In [242]:
beta = lc_pd_dataframe(fields="decay_rads", nuclides=nuc, rad_types="bm")

In [243]:
betaplus = lc_pd_dataframe(fields="decay_rads", nuclides=nuc, rad_types="bp")

In [244]:
binbeta = lc_pd_dataframe(fields="bin_beta", nuclides=nuc, rad_types="bm")

In [245]:
gamma = lc_pd_dataframe(fields="decay_rads", nuclides=nuc, rad_types="g")

In [246]:
alpha.columns

Index(['0'], dtype='object')

In [247]:
beta.columns

Index(['mean_energy', 'unc_me', 'intensity_beta', 'unc_ib',
       'daughter_level_hl', 'daughter_level_energy', 'max_energy', 'unc_me.1',
       'log_ft', 'unc_lf', 'transition_type', 'anti_nu_mean_energy', 'unc_ame',
       'p_z', 'p_n', 'p_symbol', 'p_energy_shift', 'p_energy', 'unc_pe', 'jp',
       'half_life', 'operator_hl', 'unc_hl', 'unit_hl', 'half_life_sec',
       'unc_hls', 'decay', 'decay_%', 'unc_d', 'q', 'unc_q', 'd_z', 'd_n',
       'd_symbol', 'ensdf_publication_cut-off', 'ensdf_authors',
       'Extraction_date'],
      dtype='object')

In [237]:
betaplus.columns

Index(['0'], dtype='object')

In [238]:
betaplus

,0


In [239]:
gamma.columns

Index(['0'], dtype='object')

In [191]:
gamma[["energy", "intensity", "decay", "p_energy", "p_symbol", "d_symbol"]]

KeyError: "None of [Index(['energy', 'intensity', 'decay', 'p_energy', 'p_symbol', 'd_symbol'], dtype='object')] are in the [columns]"

In [177]:
binbeta.columns

Index(['p_z', 'p_n', 'p_symbol', 'p_energy', 'd_z', 'd_n', 'd_symbol',
       'bin_en', 'dn_de', 'unc_dn_de', 'dn_de_nu', 'unc_dn_de_nu',
       'extraction_date'],
      dtype='object')

In [178]:
binbeta

,p_z,p_n,p_symbol,p_energy,d_z,d_n,d_symbol,bin_en,dn_de,unc_dn_de,dn_de_nu,unc_dn_de_nu,extraction_date


In [321]:
heaviest_ground_states = all_ground_states.query("abundance > 0").sort_values("n", ascending=False).drop_duplicates("z")

In [341]:
for heav in heaviest_ground_states.itertuples():
    z = heav.z - 1
    n = heav.n + 1
    try:
        signal = all_ground_states.query(f"z == {z} & n == {n}").iloc[0]
    except IndexError:
        continue
    
    if signal.half_life_sec < 60*60:
        continue
    
    nuc = f"{signal.n + signal.z}{signal.symbol}"
    
    gammas = lc_pd_dataframe(fields="decay_rads", nuclides=nuc, rad_types="g")
    try:
        maxg = gammas.intensity.max()
    except AttributeError:
        continue
        
    if maxg < 10:
        continue
        
    print(nuc, maxg)

231Th 56.0
197Pt 45.8869353095
187W 33.2
181Hf 80.5
176Yb 93.0
165Dy 14.24542455
159Gd 35.7
153Sm 50.1
142La 47.4
141Ce 48.3
139Ba 23.7249
133Xe 36.9
127Te 18.869303630720506
123Sn 85.69
115Cd 46.02
109Pd 56.0
103Ru 91.0
87Kr 49.6
75Ge 39.5
65Ni 23.59
59Fe 56.5
41Ar 99.16
